In [1]:
import os
project_path = "C:/workspace/Bus Project"
os.chdir(project_path)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import folium
import datetime
import random

import bus.analyzer as anz

In [63]:
cluster_df = anz.load_cluster_df()
length = len(cluster_df)
a = cluster_df.loc[random.randrange(0,length)]
b = cluster_df.loc[random.randrange(0,length)]

In [64]:
def set_dist(cluster_df, x1, y1, x2, y2):
    selector = list(cluster_df.columns)
    cluster_df['dist1_x'] = (cluster_df['cluster_longitude'] - x1)**2
    cluster_df['dist1_y'] = (cluster_df['cluster_latitude'] - y1)**2
    cluster_df['dist2_x'] = (cluster_df['cluster_longitude'] - x2)**2
    cluster_df['dist2_y'] = (cluster_df['cluster_latitude'] - y2)**2

    cluster_df['dist1'] = (cluster_df['dist1_x'] + cluster_df['dist1_y'])**(1/2)
    cluster_df['dist2'] = (cluster_df['dist2_x'] + cluster_df['dist2_y'])**(1/2)
    cluster_df['dist'] = cluster_df['dist1'] + cluster_df['dist2']
    cluster_df['dist'] = cluster_df['dist']*6500000/360
    cluster_df['dist'] = cluster_df['dist'].apply(lambda x : int(x))
    selector.append("dist")
    cluster_df = cluster_df[selector]
    cluster_df = cluster_df.sort_values(by="dist")
    return cluster_df

def dist(x1, y1, x2, y2):
    return ((x1-x2)**2+(y1-y2)**2)**(1/2)*6500000/360

x1 = a[3]
y1 = a[4]
x2 = b[3]
y2 = b[4]
d = dist(x1, y1, x2, y2)
df = set_dist(cluster_df, x1, y1, x2, y2)
df = df[df['dist'] <= d*1.05]

In [65]:
def draw_cluster(df,x1, y1, x2, y2):
    center = [df["cluster_latitude"].mean(), df["cluster_longitude"].mean()]
    map = folium.Map(location=center, zoom_start=10)
    
    for i in df.index:
        folium.CircleMarker([df.loc[i, "cluster_latitude"], df.loc[i, "cluster_longitude"]], color = 'blue', weight = 3, radius=1).add_to(map)

    return map

map = draw_cluster(df, x1, y1, x2, y2)
folium.CircleMarker([y1, x1], color = 'red', weight = 10, radius=3).add_to(map)
folium.CircleMarker([y2, x2], color = 'red', weight = 10, radius=3).add_to(map)
map